In [ ]:
import numpy as np
import pandas as pd
import gc
from joblib import dump,load
pd.set_option('display.max_columns', None)

import tensorflow as tf
import tensorflow_addons as tfa
from tqdm.notebook import tqdm

In [ ]:
path='../input/jane-street-tf-lamb/'

In [ ]:
# model parameters
class Params: pass
params=load(path+'params.joblib')
for x in dir(params):
    if not x.startswith('__'):
        print(f'{x}: {params.__getattribute__(x)}')

In [ ]:
features=[f'feature_{i}' for i in range(130)]
y_names=['resp', 'resp_1', 'resp_2', 'resp_3', 'resp_sum']
nan_cols=['feature_16','feature_127','feature_117','feature_125','feature_92',
             'feature_55','feature_121','feature_12','feature_22','feature_78',
             'feature_8','feature_28']
nan_num=[int(x.split('_')[1]) for x in nan_cols]
nan_names=[f'nan_{i}' for i in nan_num]
lag_cols=['half_of_day','time','lag_64']
lag_cols+=[f'lag_{i*params.lag}' for i in range(1,params.n_lags)]
x_names=features+lag_cols+nan_names

In [ ]:
means=load(path+'means.joblib')
stds=load(path+'stds.joblib')

In [ ]:
def create_model(n_in, n_out, layers, dropout_rate, optimizer, metrics):
    
    inp = tf.keras.layers.Input(shape = (n_in, ))  
        
    x=inp
    for i,hidden_units in enumerate(layers): 
        x = tf.keras.layers.BatchNormalization()(x)
        if i>0:   
            x = tf.keras.layers.Dropout(dropout_rate)(x)    
        else: 
            x = tf.keras.layers.Dropout(.01)(x)    
        x = tf.keras.layers.Dense(hidden_units)(x)
        x = tf.keras.layers.Activation('relu')(x)
        
    x = tf.keras.layers.Dense(n_out)(x)
    out = tf.keras.layers.Activation('sigmoid')(x)
    
    model = tf.keras.models.Model(inputs = inp, outputs = out)
    model.compile(optimizer = optimizer,
                  loss = tf.keras.losses.BinaryCrossentropy(), 
                  metrics = metrics, 
#                   run_eagerly=True
                 )
    
    return model

In [ ]:
optimizer=tfa.optimizers.LAMB(learning_rate=params.lr,weight_decay_rate=params.wd)

In [ ]:
class LiteModel:

    @classmethod
    def from_keras_model(cls, kmodel):
        converter = tf.lite.TFLiteConverter.from_keras_model(kmodel)
        tflite_model = converter.convert()
        return LiteModel(tf.lite.Interpreter(model_content=tflite_model))

    def __init__(self, interpreter):
        self.interpreter = interpreter
        self.interpreter.allocate_tensors()
        input_det = self.interpreter.get_input_details()[0]
        output_det = self.interpreter.get_output_details()[0]
        self.input_index = input_det["index"]
        self.output_index = output_det["index"]
        self.input_shape = input_det["shape"]
        self.output_shape = output_det["shape"]
        self.input_dtype = input_det["dtype"]
        self.output_dtype = output_det["dtype"]
        
    def predict(self, inp):
        inp = inp.astype(self.input_dtype)
        count = inp.shape[0]
        out = np.zeros((count, self.output_shape[1]), dtype=self.output_dtype)
        for i in range(count):
            self.interpreter.set_tensor(self.input_index, inp[i:i+1])
            self.interpreter.invoke()
            out[i] = self.interpreter.get_tensor(self.output_index)[0]
        return out

    def predict_single(self, inp):
        """ Like predict(), but only for a single record. The input data can be a Python list. """
        inp = np.array([inp], dtype=self.input_dtype)
        self.interpreter.set_tensor(self.input_index, inp)
        self.interpreter.invoke()
        out = self.interpreter.get_tensor(self.output_index)
        return out[0]

In [ ]:
models=[]
for i in range(params.n_runs):
    for j in range(params.n_folds):   
        model=create_model(len(x_names), len(y_names), params.layers, params.dropout_rate, 
                   optimizer=optimizer, metrics=[])
        model.load_weights(path+f'models/saved_model_{i}_{j}.hdf5')
        models.append(model)

In [ ]:
lite_models=[]
for i in range(params.n_runs):
    for j in range(params.n_folds):   
        model=create_model(len(x_names), len(y_names), params.layers, params.dropout_rate, 
           optimizer=optimizer, metrics=[])
        model.load_weights(path+f'models/saved_model_{i}_{j}.hdf5')
        lite_model = LiteModel.from_keras_model(model)
        lite_models.append(lite_model)

In [ ]:
def fill_na(array,means):
    if np.isnan(array.sum()):
        array = np.nan_to_num(array) + np.isnan(array) * means
    return array

In [ ]:
test_df=pd.read_csv('../input/jane-street-market-prediction/example_test.csv',nrows=1)

In [ ]:
test_df[lag_cols]=0

In [ ]:
test_df[features+lag_cols]

In [ ]:
means=means.values[None,:]
stds=stds.values[None,:]

In [ ]:
#compare keras vs TF Lite results
feats=fill_na(test_df[features].values,means[:,:len(features)])
lags=test_df[lag_cols].values
nans=test_df[nan_cols].isnull().values.astype('float32')
X=np.concatenate((feats,lags,nans),1)
X=(X-means)/stds
for model,lite_model in zip(models,lite_models):
    preds=model(X,training=False)[0].numpy()
    lite_preds=lite_model.predict_single(X.squeeze())
    assert np.allclose(preds,lite_preds)

In [ ]:
import janestreet
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

In [ ]:
votes_per_model=len(y_names)
prev_date=-1
prev_half=-1.
ts_id=-1
latest_feat_0=np.zeros(params.lag*(params.n_lags-1))
latest_grad_64=np.zeros(params.grad_64_lag)

In [ ]:
MAX_NEG_VOTES=(len(lite_models)*len(y_names))/2
for (test_df, pred_df) in tqdm(iter_test):

    weight=test_df['weight'].item()
    
    feat_0=test_df['feature_0'].item()
    latest_feat_0=np.concatenate((latest_feat_0,np.array([feat_0])))[1:]
    
    date=test_df['date'].item()
    ts_id+=1
    feat_64=test_df['feature_64'].item()
    half_of_day=float(feat_64>1)
    if date!=prev_date or half_of_day!=prev_half:
        min_ts=ts_id
        grad_64=0.
    else:
        grad_64=feat_64-prev_64   
    latest_grad_64=np.concatenate((latest_grad_64,np.array([grad_64])))[1:]    
    
    time=ts_id-min_ts
    lag_64=latest_grad_64.sum()
    
    prev_date=date
    prev_half=half_of_day
    prev_64=feat_64
    
    extra_feats=np.array((half_of_day, time, lag_64))[None,:]
    
    
    if weight > 0:   
        
        lags=latest_feat_0.reshape(params.n_lags-1,-1).sum(1)[::-1].cumsum()[None,:]
        feats=fill_na(test_df[features].values,means[:,:len(features)])
        nans=test_df[nan_cols].isnull().values.astype('float32')
        
        X=np.concatenate((feats,extra_feats,lags,nans),1)
        X=(X-means)/stds
        
        total_negative_votes=0
           
        pred_df.action = 1

        for lite_model in lite_models:   
            pred=lite_model.predict_single(X.squeeze())
            negative_votes=(pred<0.5).sum() * (max(1,weight)**0.2)
            total_negative_votes+=negative_votes
            if total_negative_votes>MAX_NEG_VOTES:
                pred_df.action = 0
                break
    
    else:
        pred_df.action = 0
        
    env.predict(pred_df)

In [ ]:
sub=pd.read_csv('submission.csv')

In [ ]:
sub.head(10)

In [ ]:
sub.action.value_counts()